## Classification Problem - Cross Validation - PySpark Local

### Prepare the Data

First, import the libraries that we will need and prepare the training and test data:

In [ ]:
# Import libraries
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.context import SparkContext, SparkConf
from pyspark.sql.session import SparkSession

import numpy as np

In [1]:
# PySpark session initialization
conf = SparkConf().setAppName("HPC PySpark App")
sc = SparkContext('local', conf=conf)
spark = SparkSession(sc)

# Load the source data
csv = spark.read.csv('../../data/bank.csv', inferSchema=True, header=True, sep=';')

# Select features and label
data = csv.select(*csv.columns[:-1], (col("y").alias("label")))
print(data)

# Split the data 70-30
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")

DataFrame[age: double, job: double, marital: double, education: double, default: double, balance: double, housing: double, loan: double, contact: double, day: double, month: double, duration: double, campaign: double, pdays: double, previous: double, poutcome: double, label: double]


### Define the Pipeline
Now define a pipeline that creates a feature vector and trains a classification model

In [2]:
# Define the pipeline
assembler = VectorAssembler(inputCols = data.columns[:-1], outputCol="features")
print("Input Columns: ", assembler.getInputCols())
print("Output Column: ", assembler.getOutputCol())

lr = LogisticRegression(labelCol="label", featuresCol="features")
pipeline = Pipeline(stages=[assembler, lr])

Input Columns:  ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome']
Output Column:  features


### Tune Parameters
You can tune parameters to find the best model for your data. A simple way to do this is to use  **TrainValidationSplit** to evaluate each combination of parameters defined in a **ParameterGrid** against a subset of the training data in order to find the best performing parameters.

#### Regularization Params and iterations
Save Regularization params and Max iteration in order to pass them at the paramGrid

In [3]:
print(lr.explainParams())
lr_reg_params = np.logspace(-5, 5, 7)
lr_max_iter = np.linspace(1, 10, 2)

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The

#### CrossValidation
Set CrossValidation estimator, evaluator, params and folds

In [4]:
folds = 4
evaluator=BinaryClassificationEvaluator()
paramGrid = ParamGridBuilder().addGrid(lr.regParam, lr_reg_params).addGrid(lr.maxIter, lr_max_iter).build()

cv = CrossValidator(estimator=pipeline, evaluator=evaluator, estimatorParamMaps=paramGrid, numFolds=folds)

#### Training
Starting and ending time for the fit

In [5]:
import time
tic = time.time()

model = cv.fit(train)

toc = time.time()
print("Elapsed time ", toc-tic)

Elapsed time  76.744393825531


### Test the Model
Now you're ready to apply the model to the test data.

In [6]:
prediction = model.transform(test)
predicted = prediction.select("features", "prediction", "probability", "trueLabel")
# print(*predicted.select('prediction', 'trueLabel').collect(), sep='\n')

### Compute Confusion Matrix Metrics
Classifiers are typically evaluated by creating a *confusion matrix*, which indicates the number of:
- True Positives
- True Negatives
- False Positives
- False Negatives

From these core measures, other evaluation metrics such as *precision* and *recall* can be calculated.

In [7]:
print('true positives:', predicted.filter('trueLabel == 1').count())
print('true negatives:', predicted.filter('trueLabel == 0').count())

true positives: 1588
true negatives: 12088


In [8]:
tp = float(predicted.filter("prediction == 1.0 AND trueLabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND trueLabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND trueLabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND trueLabel == 1").count())
correctly_classified = (tp+tn) / predicted.count()

In [9]:
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", tp / (tp + fp)),
 ("Recall", tp / (tp + fn)),
 ("Correctly Classified", correctly_classified)],["metric", "value"])
metrics.show()

+--------------------+------------------+
|              metric|             value|
+--------------------+------------------+
|                  TP|             383.0|
|                  FP|             210.0|
|                  TN|           11878.0|
|                  FN|            1205.0|
|           Precision|0.6458684654300169|
|              Recall| 0.241183879093199|
|Correctly Classified|0.8965340742907283|
+--------------------+------------------+

